In [119]:
import os
import pandas as pd
import numpy as np
import math
from datetime import datetime
import datetime as dt
import tensorflow as tf
from tensorflow import keras
from keras.layers import GRU, Dropout, SimpleRNN, LSTM, Dense, SimpleRNN, GRU
from keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn

import plotly as py
import plotly.io as pio

# Mute sklearn warnings
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
simplefilter(action='ignore', category=DeprecationWarning)

from plotly.offline import init_notebook_mode

# Show charts when running kernel
init_notebook_mode(connected=True)

from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score 
from sklearn.metrics import mean_poisson_deviance, mean_gamma_deviance, accuracy_score
from sklearn.preprocessing import MinMaxScaler

from itertools import cycle
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import seaborn as sns
import matplotlib.pyplot as plt
from colorama import Fore

In [120]:
# Path to your CSV file
csv_file_path = '/Users/rayhanzirvi/Desktop/ycom_2010_2023.csv'
# csv_file_path = 'math-ysics/SpringData2024/data/consume-produce_data/annual_generation_state.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(csv_file_path)

# for col in df.columns:
#     print(col)

# Display the first few rows of the DataFrame
print(df.head())

# df_good = df[['State', '2021']]

# print(df_2021)

  geotype  geoid     geoname        varname  x2010  x2011  x2012  x2013  \
0   state      1     Alabama  affectweather    NaN    NaN    NaN    NaN   
1   state      2      Alaska  affectweather    NaN    NaN    NaN    NaN   
2   state      4     Arizona  affectweather    NaN    NaN    NaN    NaN   
3   state      5    Arkansas  affectweather    NaN    NaN    NaN    NaN   
4   state      6  California  affectweather    NaN    NaN    NaN    NaN   

   x2014  x2015  x2016  x2017  x2018  x2019  x2020  x2021  x2022  x2023  
0    NaN    NaN    NaN    NaN  52.66  49.22  53.33  53.64  49.63  50.28  
1    NaN    NaN    NaN    NaN  57.48  54.94  60.92  62.25  59.17  60.66  
2    NaN    NaN    NaN    NaN  60.32  58.40  63.55  64.22  61.73  63.12  
3    NaN    NaN    NaN    NaN  53.17  50.78  55.72  55.89  51.26  52.43  
4    NaN    NaN    NaN    NaN  67.21  64.51  69.28  70.14  67.47  68.39  


In [121]:
valid_state_names = {
    'Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado',
    'Connecticut', 'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Idaho',
    'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana',
    'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota',
    'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada',
    'New Hampshire', 'New Jersey', 'New Mexico', 'New York',
    'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon',
    'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota',
    'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
    'West Virginia', 'Wisconsin', 'Wyoming', 'District of Columbia'
}

In [122]:
df_drill = df[df['varname'] == 'drilloffshore']
# Now, drop duplicates based on 'State', keeping only the first occurrence

df_drill = df_drill[['geoname', 'x2021']]

df_drill = df_drill[df_drill['geoname'].isin(valid_state_names)]

df_drill.head()

,geoname,x2021
918,Alabama,61.39
919,Alaska,48.40
920,Arizona,52.80
921,Arkansas,55.77
922,California,39.44


In [123]:
df_reg = df[df['varname'] == 'regulate']
# Now, drop duplicates based on 'State', keeping only the first occurrence

df_reg = df_reg[['geoname', 'x2021']]

df_reg = df_reg[df_reg['geoname'].isin(valid_state_names)]

df_reg.head()

,geoname,x2021
3009,Alabama,67.15
3010,Alaska,69.14
3011,Arizona,71.21
3012,Arkansas,67.11
3013,California,74.12


In [124]:
df_combined = pd.merge(df_drill, df_reg, on=['geoname'], suffixes=('_drill', '_co2'))

df_combined.head()

,geoname,x2021_drill,x2021_co2
0,Alabama,61.39,67.15
1,Alaska,48.40,69.14
2,Arizona,52.80,71.21
3,Arkansas,55.77,67.11
4,California,39.44,74.12


In [125]:
df_combined.rename(columns={'geoname': 'State'}, inplace=True)
df_combined.rename(columns={'x2021_drill': 'Drill'}, inplace=True)
df_combined.rename(columns={'x2021_co2': 'co2'}, inplace=True)

In [126]:
# State name to abbreviation mapping
state_abbreviations = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR',
    'California': 'CA', 'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE',
    'Florida': 'FL', 'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID',
    'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS',
    'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD',
    'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS',
    'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV',
    'New Hampshire': 'NH', 'New Jersey': 'NJ', 'New Mexico': 'NM', 'New York': 'NY',
    'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK',
    'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC',
    'South Dakota': 'SD', 'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT',
    'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA', 'West Virginia': 'WV',
    'Wisconsin': 'WI', 'Wyoming': 'WY', 'District of Columbia': 'DC',
}

# Use map() to change state names to abbreviations
df_combined['State'] = df_combined['State'].map(state_abbreviations)

df_combined.head()

,State,Drill,co2
0,AL,61.39,67.15
1,AK,48.40,69.14
2,AZ,52.80,71.21
3,AR,55.77,67.11
4,CA,39.44,74.12


In [127]:
sum_drill = df_combined['Drill'].sum()
sum_co2 = df_combined['co2'].sum()

average_drill = sum_drill / len(df_combined)
average_co2 = sum_co2 / len(df_combined)

# Adding the new row with average values
# new_row = {'State': 'Average', 'Drill': average_drill, 'co2': average_co2}
# df_combined = df_combined.append(new_row, ignore_index=True)

df_combined.loc[len(df_combined)] = ['Average', average_drill, average_co2]

df_combined.tail()

,State,Drill,co2
47,WA,38.620000,73.000000
48,WV,58.180000,62.750000
49,WI,52.420000,72.360000
50,WY,59.120000,62.290000
51,Average,50.627647,70.892157


In [128]:
# Export DataFrame to CSV
df_combined.to_csv('/Users/rayhanzirvi/Desktop/sentiment_regress.csv', index=False)